In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import random
from scipy import ndimage
import tensorflow_hub as hub

emociones = pd.read_csv("./icml_face_data.csv", delimiter=",")
emociones.columns = emociones.columns.str.strip()

emociones = emociones.iloc[1:21]
emociones.pixels = emociones.pixels.apply(lambda x: np.array(x.split()).reshape(48,48).astype(float))/255


def transformation(x):
    rotate_rand = random.randint(-30,30)
    x = ndimage.rotate(x, rotate_rand, reshape = False)

    bright_rand = random.uniform(1.5, 2)
    x = np.clip(x * bright_rand, 0.0, 255.0)

    return x

emociones.pixels = emociones.pixels.apply(lambda x: cv2.resize(x, (224,224)))
emociones_augmented = emociones.copy()

for n in range(len(emociones_augmented.pixels)):
    emociones_augmented.pixels.iloc[n] = transformation(emociones.pixels.iloc[n])


result = pd.concat([emociones, emociones_augmented])

# plt.imshow(result.iloc[10].pixels, cmap='gray')
# plt.show()
# plt.imshow(result.iloc[30].pixels, cmap='gray')
# plt.show()

resnet_url = "https://www.kaggle.com/models/tensorflow/resnet-50/frameworks/TensorFlow2/variations/feature-vector/versions/1"
IMAGE_SHAPE = (224, 224)
COUNT_UNIQUE_EMOTIONS = len(pd.unique(result.emotion))

feature_extractor_layer = hub.KerasLayer(resnet_url,
                                           trainable=False, # congela los patrones subyacentes
                                           name='feature_extraction_layer',
                                           input_shape=IMAGE_SHAPE+(1,)) # define la forma de la imagen de entrada

model = tf.keras.Sequential([
    feature_extractor_layer, # utiliza la capa de extracción de características como la base
    tf.keras.layers.Dense(COUNT_UNIQUE_EMOTIONS, activation='softmax', name='output_layer') # crea nuestra propia capa de salida
])



<ipython-input-11-ca2f5073b6e1>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emociones_augmented.pixels.iloc[n] = transformation(emociones.pixels.iloc[n])


ValueError: Exception encountered when calling layer "feature_extraction_layer" (type KerasLayer).

in user code:

    File "/usr/local/lib/python3.10/dist-packages/tensorflow_hub/keras_layer.py", line 250, in call  *
        result = smart_cond.smart_cond(training,

    ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
      Positional arguments (3 total):
        * <tf.Tensor 'inputs:0' shape=(None, 224, 224, 1) dtype=float32>
        * False
        * None
      Keyword arguments: {}
    
     Expected these arguments to match one of the following 4 option(s):
    
    Option 1:
      Positional arguments (3 total):
        * TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1')
        * False
        * None
      Keyword arguments: {}
    
    Option 2:
      Positional arguments (3 total):
        * TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1')
        * True
        * None
      Keyword argu

In [ ]:
# Resnet 50 V2 feature vector
resnet_url = "https://www.kaggle.com/models/tensorflow/resnet-50/frameworks/TensorFlow2/variations/feature-vector/versions/1"


# Original: EfficientNetB0 feature vector (version 1)
efficientnet_url="https://www.kaggle.com/models/tensorflow/efficientnet/frameworks/TensorFlow2/variations/b0-feature-vector/versions/1"

def create_model(model_url, num_classes=10):
  """Toma una URL de TensorFlow Hub y crea un modelo secuencial de Keras con ella.

  Args:
    model_url (str): Una URL de extracción de características de TensorFlow Hub.
    num_classes (int): Número de neuronas de salida en la capa de salida,
      debe ser igual al número de clases objetivo, por defecto 10.

  Returns:
    Un modelo secuencial de Keras sin compilar con model_url como capa
    de extracción de características y capa de salida densa con num_classes salidas.
  """
  # Descarga el modelo preentrenado y guárdalo como una capa de Keras
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False, # congela los patrones subyacentes
                                           name='feature_extraction_layer',
                                           input_shape=IMAGE_SHAPE+(3,)) # define la forma de la imagen de entrada

  # Crea nuestro propio modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    feature_extractor_layer, # utiliza la capa de extracción de características como la base
    tf.keras.layers.Dense(num_classes, activation='softmax', name='output_layer') # crea nuestra propia capa de salida
  ])

  return model

model = create_model(resnet_url, 4)
model.compile(loss='categorical_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])



In [ ]:
fotos = result.pixels
fotos
# model.fit(fotos, epochs=5)
# model

1     [[0.592156862745098, 0.592156862745098, 0.5920...
2     [[0.09411764705882353, 0.09411764705882353, 0....
3     [[0.0784313725490196, 0.0784313725490196, 0.07...
4     [[0.30196078431372547, 0.30196078431372547, 0....
5     [[0.3333333333333333, 0.3333333333333333, 0.33...
6     [[0.11764705882352941, 0.11764705882352941, 0....
7     [[0.01568627450980392, 0.01568627450980392, 0....
8     [[0.4196078431372549, 0.4196078431372549, 0.41...
9     [[0.054901960784313725, 0.054901960784313725, ...
10    [[0.8588235294117647, 0.8588235294117647, 0.85...
11    [[0.00392156862745098, 0.00392156862745098, 0....
12    [[0.4823529411764706, 0.4823529411764706, 0.48...
13    [[0.03137254901960784, 0.03137254901960784, 0....
14    [[0.9882352941176471, 0.9882352941176471, 0.98...
15    [[0.8784313725490196, 0.8784313725490196, 0.87...
16    [[0.6352941176470588, 0.6352941176470588, 0.64...
17    [[0.9254901960784314, 0.9254901960784314, 0.92...
18    [[0.8235294117647058, 0.8235294117647058, 

In [ ]:
from keras.models import Sequential
#Import from keras_preprocessing not from keras.preprocessing
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

traindf=pd.read_csv("./icml_face_data.csv",dtype=str)
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

datagen

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import random
from scipy import ndimage
import tensorflow_hub as hub

emociones = pd.read_csv("./icml_face_data.csv", delimiter=",")
emociones.columns = emociones.columns.str.strip()

emociones = emociones.iloc[1:21]
emociones.pixels = emociones.pixels.apply(lambda x: np.array(x.split()).reshape(48,48).astype(float))/255

def transformation(x):
    rotate_rand = random.randint(-30,30)
    x = ndimage.rotate(x, rotate_rand, reshape = False)

    bright_rand = random.uniform(1.5, 2)
    x = np.clip(x * bright_rand, 0.0, 255.0)

    return x

emociones.pixels = emociones.pixels.apply(lambda x: cv2.resize(x, (224,224)))
emociones_augmented = emociones.copy()

for n in range(len(emociones_augmented.pixels)):
    emociones_augmented.pixels.iloc[n] = transformation(emociones.pixels.iloc[n])

result = pd.concat([emociones, emociones_augmented])

# Convert grayscale images to RGB
result['pixels_rgb'] = result['pixels'].apply(lambda x: cv2.cvtColor((x * 255).astype(np.uint8), cv2.COLOR_GRAY2RGB))

resnet_url = "https://www.kaggle.com/models/tensorflow/resnet-50/frameworks/TensorFlow2/variations/feature-vector/versions/1"
IMAGE_SHAPE = (224, 224)
COUNT_UNIQUE_EMOTIONS = len(pd.unique(result.emotion))

feature_extractor_layer = hub.KerasLayer(resnet_url,
                                           trainable=False, # Freeze underlying patterns
                                           name='feature_extraction_layer',
                                           input_shape=IMAGE_SHAPE+(3,)) # Change input shape to (224, 224, 3)

model = tf.keras.Sequential([
    feature_extractor_layer, # Use the feature extraction layer as the base
    tf.keras.layers.Dense(COUNT_UNIQUE_EMOTIONS, activation='softmax', name='output_layer') # Create our own output layer
])


<ipython-input-3-4385ade50856>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emociones_augmented.pixels.iloc[n] = transformation(emociones.pixels.iloc[n])


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import tensorflow_hub as hub

# Función para crear el modelo ResNet50 con capas adicionales y más regularización
def create_model_with_extra_layers_and_regularization(resnet_url, num_classes):
    feature_extractor_layer = hub.KerasLayer(resnet_url,
                                             trainable=False,
                                             name='feature_extraction_layer',
                                             input_shape=(224, 224, 3))

    model = tf.keras.Sequential([
        feature_extractor_layer,
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),  # Aumento de la tasa de Dropout
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),  # Aumento de la tasa de Dropout
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    return model

# Normalización de píxeles
X = np.array(result['pixels_rgb'].tolist()) / 255.0

# Codificación de etiquetas
y = pd.get_dummies(result['emotion']).values

# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Aumento de datos
datagen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, height_shift_range=0.1,
                             shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
datagen.fit(X_train)

# URL del modelo ResNet50
resnet_url = "https://www.kaggle.com/models/tensorflow/resnet-50/frameworks/TensorFlow2/variations/feature-vector/versions/1"

# Crear el modelo ResNet50 con capas adicionales y más regularización
resnet_model = create_model_with_extra_layers_and_regularization(resnet_url, num_classes=y_train.shape[1])

# Compilar el modelo
resnet_model.compile(loss='categorical_crossentropy',
                     optimizer=tf.keras.optimizers.Adam(),
                     metrics=['accuracy'])

# Entrenar el modelo
resnet_history = resnet_model.fit(datagen.flow(X_train, y_train, batch_size=32),
                                  epochs=15,
                                  steps_per_epoch=len(X_train) // 32,
                                  validation_data=(X_test, y_test),
                                  callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                                                         experiment_name="resnet50V2_with_extra_layers_and_regularization")])


Saving TensorBoard log files to: tensorflow_hub/resnet50V2_with_extra_layers_and_regularization/20240310-163836
Epoch 1/15
1/1 [==============================] - 25s 25s/step - loss: 2.4071 - accuracy: 0.0938 - val_loss: 1.3263 - val_accuracy: 0.2500
Epoch 2/15
1/1 [==============================] - 11s 11s/step - loss: 3.0217 - accuracy: 0.0938 - val_loss: 1.1391 - val_accuracy: 0.5000
Epoch 3/15
1/1 [==============================] - 10s 10s/step - loss: 2.0008 - accuracy: 0.3438 - val_loss: 1.1441 - val_accuracy: 0.5000
Epoch 4/15
1/1 [==============================] - 7s 7s/step - loss: 1.4965 - accuracy: 0.4062 - val_loss: 1.2126 - val_accuracy: 0.5000
Epoch 5/15
1/1 [==============================] - 10s 10s/step - loss: 1.8652 - accuracy: 0.4375 - val_loss: 1.2683 - val_accuracy: 0.3750
Epoch 6/15
1/1 [==============================] - 8s 8s/step - loss: 1.7005 - accuracy: 0.3750 - val_loss: 1.2860 - val_accuracy: 0.3750
Epoch 7/15
1/1 [==============================] - 10s 10s/